# **AutoGluon-Text-Classificationを用いたテキスト分類**

## **概要**
このノートブックでは、AutoGluon-Text-Classificationを用いて日本語のAmazonの商品レビューに対する感情分析を行います。
つまり、そのレビューが Positive (Rating が 5 or 4) か、Negative (Rating が 1 or 2)なのかを判定します。
これは文書を Positive か Negative に分類する2クラスの分類問題として扱うことができます。

AutoGluon-Text-Classificationは、データを用意し、学習時にパラメータを設定するだけでモデルを作成することができます。
2020.8時点では、Text-Classificationにはmodelのsave/loadが実装されていません。
そこで今回はpickleを用いてモデルのsave/loadを行います。
学習には多くのメモリを必要としますので、多くのメモリを利用できるインスタンスを利用してください。

## **データの取得**

Amazon の商品レビューデータセットは [Registry of Open Data on AWS](https://registry.opendata.aws/) で公開されており、 
以下からダウンロード可能です。
このノートブックでは、日本語のデータセットをダウンロードします。
- データセットの概要  
https://registry.opendata.aws/amazon-reviews/

- 日本語のデータセット(readme.htmlからたどることができます）  
https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_JP_v1_00.tsv.gz

以下では、データをダウンロードして解凍 (unzip) します。

In [ ]:
import urllib.request
import os
import gzip
import shutil

download_url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_JP_v1_00.tsv.gz" 
dir_name = "data"
file_name = "amazon_review.tsv.gz"
tsv_file_name = "amazon_review.tsv"
file_path = os.path.join(dir_name,file_name)
tsv_file_path = os.path.join(dir_name,tsv_file_name)

os.makedirs(dir_name, exist_ok=True)

if os.path.exists(file_path):
    print("File {} already exists. Skipped download.".format(file_name))
else:
    urllib.request.urlretrieve(download_url, file_path)
    print("File downloaded: {}".format(file_path))
    
if os.path.exists(tsv_file_path):
    print("File {} already exists. Skipped unzip.".format(tsv_file_name))
else:
    with gzip.open(file_path, mode='rb') as fin:
        with open(tsv_file_path, 'wb') as fout:
            shutil.copyfileobj(fin, fout)
            print("File uznipped: {}".format(tsv_file_path))

## **データの前処理**

ダウンロードしたデータには学習に不要なデータや直接利用できないデータもあります。以下の前処理で利用できるようにします。

1. ダウンロードしたデータには不要なデータも含まれているので削除し、2クラス分類 (positive が 1, negative が 0)となるように評価データを加工します。
2. 学習データ、テストデータに分けて、保存します。

### **データの加工**

今回利用しないデータは以下の2つです。必要なデータだけ選んで保存します。
- 評価データ `star_rating` と レビューのテキストデータ `review_body` 以外のデータ
- 評価が 3 のデータ (positive でも negative でもないデータ)

また、評価が1, 2 のデータはラベル 0 (negative) に、評価が4, 5 のデータはラベル 1 (positive) にします。

In [ ]:
import pandas as pd
df = pd.read_csv(tsv_file_path, sep ='\t')
df_pos_neg = df.loc[:, ["star_rating", "review_body"]]
df_pos_neg = df_pos_neg[df_pos_neg.star_rating != 3]
df_pos_neg.loc[df_pos_neg.star_rating < 3, "star_rating"] = 0
df_pos_neg.loc[df_pos_neg.star_rating > 3, "star_rating"] = 1

2020.8 時点では、AutoGluon-Text は csv のみサポートしています。そのため、カンマを含む文章を対象とする際は、文章中のカンマと、文章とラベルを区別するカンマの見分けが付きません。そこで、カンマを含む文章のみをダブルクォーテーション(")で囲みます。文中にダブルクォーテーションが含まれると正常に処理されないため、まずはそれを除いてから、カンマを含む文章のみにダブルクォーテーションを付与します。

In [ ]:
df_pos_neg["review_body"] = df_pos_neg["review_body"].str.replace("\"", "")

rows_incl_comma = df_pos_neg["review_body"].str.contains(",")
df_pos_neg.loc[rows_incl_comma, "review_body"] = "\"" + df_pos_neg.loc[rows_incl_comma,"review_body"] +"\""

### **データの分割**
すべてのデータを学習データとすると、データを使って作成したモデルが良いのか悪いのか評価するデータが別途必要になります。 そこで、データを学習データ、テストデータに分割して利用します。学習データはモデルの学習に利用し、最終的に作成されたモデルに対してテストデータによる評価を行います。

train_ratio で設定した割合のデータを学習データとし、残ったデータをテストデータに分割して利用します。

In [ ]:
import numpy as np

dir_name = "data"
file_name = "amazon_review.tsv.gz"
tsv_file_name = "amazon_review.tsv"
file_path = os.path.join(dir_name,file_name)
tsv_file_path = os.path.join(dir_name,tsv_file_name)

# Swap positions of "review_body","star_rating" because transform.py requires this order.
labeled_df = df_pos_neg.loc[:, ["review_body","star_rating"]]
data_size = len(labeled_df.index)
train_ratio = 0.9
train_index = np.random.choice(data_size, int(data_size*train_ratio), replace=False)
test_index = np.setdiff1d(np.arange(data_size), train_index)

np.savetxt('train.csv',labeled_df.iloc[train_index].values, fmt="%s,%i") 
np.savetxt('test.csv',labeled_df.iloc[test_index].values, fmt="%s,%i") 

print("Data is splitted into:")
print("Training data: {} records.".format(len(train_index)))
print("Test data: {} records.".format(len(test_index)))

### **ヘッダの追加**
AutoGluon では、データのヘッダを指定して学習することができるため、解析するテキストを`text`、ポジティブかネガティブかを表すラベルを`target`としてヘッダを与えます。

In [ ]:
!sed -i '1s/^/text,target\n/' train.csv
!sed -i '1s/^/text,target\n/' test.csv

## **学習の実行**

### **学習データのアップロード**
S3にデータをアップロードします。

In [ ]:
import sagemaker

sess = sagemaker.Session()

s3_train_data = sess.upload_data(path='train.csv', key_prefix='amazon-review-data-csv')
print("Training data is uploaded to {}".format(s3_train_data))

data_channels = {'train': s3_train_data}

### **学習ジョブの実行**
framework_version="1.6.0"を指定して下さい。
今回はGPUインスタンスであるml.p3.2xlargeを使用しますが、
ml.m4.xlargなどのCPUインスタンスでも学習可能です。
ただし、メモリが足りなくなる場合があるため、学習データの件数を減らす必要があります。

In [ ]:
from sagemaker.mxnet import MXNet

gluon_bert = MXNet("train_autogluon.py", 
                  role=sagemaker.get_execution_role(), 
                  source_dir = "src",
                  train_instance_count=1, 
                  train_instance_type="ml.p3.2xlarge",
                  framework_version="1.6.0",
                  distributions={'parameter_server': {'enabled': True}},
                  py_version = "py3",
                  hyperparameters={'batch-size': 16, 
                                   'epochs': 10, 
                                   'log-interval': 1})

gluon_bert.fit(data_channels)

## **デプロイ**
学習したモデルをデプロイします。

In [ ]:
predictor = gluon_bert.deploy(initial_instance_count=1, instance_type='ml.c5.xlarge')

## **推論**
実際の文を用いた推論を行います。
\[1\]がポジティブ、\[0\]がネガティブです。

In [ ]:
sentence = '子供でも簡単に操作ができて、暇つぶしに最高です。'
prediction = predictor.predict([sentence])
pred_label = np.argmax(prediction)
print('The input sentence sentiment is classified as [%d]. (prbability: %f)' % (pred_label, prediction[0][pred_label]))

## **評価**
モデルをダウンロードし、テストデータのaccuracyを評価します。
メモリを多く必要とするモデルを評価するため、notebookはml.c5.18xlargeなどを使用する必要があります。
また、評価の実行に数分かかります。

### **モデルのダウンロード**

In [ ]:
!aws s3 cp $gluon_bert.model_data ./
!tar zxvf model.tar.gz

### **関連するライブラリのダウンロード**

In [ ]:
!pip install --upgrade mxnet-cu100
!pip install autogluon
!pip install gluonnlp==0.8.1
!pip install cloudpickle==1.3.0

### **モデルの読み込み**

In [ ]:
import pickle
from autogluon import TextClassification as task
local_model = pickle.load(open('model', 'rb'))

### **テストセットの評価**

In [ ]:
testset = task.Dataset(filepath='test.csv', usecols=['text', 'target'])
test_acc = local_model.evaluate(testset)
print('Top-1 test acc: %.3f' % test_acc)